https://www.kaggle.com/CVxTz/keras-bidirectional-lstm-baseline-lb-0-051

In [1]:
import gc
import numpy as np
import pandas as pd
import utils

from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from tqdm import tqdm

from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling3D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: insp

Using TensorFlow backend.
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back 

/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is

In [2]:
src_sub = '/home/w/Projects/Toxic/submissions/'

s1 = pd.read_csv(src_sub + 'one_more_blend.csv')
s2 = pd.read_csv(src_sub + 'submission.csv')
s3 = pd.read_csv(src_sub + 'blend_it_all.csv')

In [3]:
s = s1.copy()
s.iloc[:, 1:] = (s1.iloc[:, 1:] + s2.iloc[:, 1:] + s3.iloc[:, 1:]) / 3
s.to_csv(src_sub + 'BlendOf3Kernels.csv', index=False)

In [2]:
def BiDirectLSTM_W2V(embedding_matrix, nb_words, embedding_dim, sequence_length):
    
    Embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[embedding_matrix],
        input_length=sequence_length,
        trainable=False)
    
    input_ = Input(shape=(sequence_length, ))
    embed_input_ = Embedding_layer(input_)
    
    x = Bidirectional(LSTM(50, return_sequences=True))(embed_input_)
    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    
    model = Model(inputs=input_, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [ ]:
train, test = utils.load_data('../data/', 'TextacyClean')

In [2]:
max_features = 200000
sequence_length = 196
embedding_dim = 300
create_embedding = False


train = pd.read_pickle("../data/train_spacy_clean.pkl")
test = pd.read_pickle("../data/test_spacy_clean.pkl")


list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_sentences_test = test["comment_text"].fillna("CVxTz").values

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index)) + 1

X_train = sequence.pad_sequences(list_tokenized_train, maxlen=sequence_length)
y_train = train[list_classes].values
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=sequence_length)

del train, test, list_tokenized_train, list_tokenized_test
gc.collect()

39

In [6]:
if create_embedding:
    embedding_file = '/home/w/Projects/Toxic/data/embeddings/wiki.en/wiki.en.vec'
    word2vec = KeyedVectors.load_word2vec_format(embedding_file, binary=False)
    print('Found %s word vectors of word2vec' % len(word2vec.vocab))

    embedding_matrix = np.zeros((nb_words, embedding_dim))
    for word, i in tqdm(word_index.items()):
        if word in word2vec.vocab:
            embedding_matrix[i] = word2vec.word_vec(word)
    print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
else:
    embedding_matrix = pd.read_pickle('../data/embeddings/FastText_300dim_embedding.pkl')

 22%|██▏       | 27846/125522 [00:00<00:00, 278404.51it/s]

Found 2519370 word vectors of word2vec


100%|██████████| 125522/125522 [00:00<00:00, 298551.54it/s]

Null word embeddings: 21796


In [7]:
pd.to_pickle(embedding_matrix, '../data/embeddings/FastText_300dim_embedding.pkl')

In [ ]:
epochs = 100
batch_size = 1024


model_callbacks = [EarlyStopping(monitor='val_loss', patience=6, verbose=1, mode='min'),
                   ReduceLROnPlateau(monitor='val_loss', factor=0.7, verbose=1,
                                     patience=4, min_lr=1e-6)]


model = BiDirectLSTM_W2V(embedding_matrix, nb_words, embedding_dim, sequence_length)
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
          validation_split=0.1, callbacks=model_callbacks)

y_test = model.predict(X_test)

In [ ]:
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = y_test
sample_submission.to_csv("../submissions/BidirectLSTM_SpacyClean.csv", index=False)